# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [3]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'
#print(ROBOT_POSE_DATA_DIR)

Populating the interactive namespace from numpy and matplotlib


In [4]:
classes = listdir(ROBOT_POSE_DATA_DIR) #listdir schreibt alle file namen von dem path auf, path ist robot_pose_data
print(classes)

['Crouch', 'Belly', 'Knee', 'Frog', 'HeadBack', 'Back', 'Left', 'StandInit', 'Stand', 'Right', 'Sit']


In [5]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE ---hier wird schon data reingelesen??
    #pickle.load benötigt als argument eine Datei
    #hatte mit wb statt rb die Knee Datei gelöscht (lokal) konnte sie mit git checkout --robot_pose_data/Knee wiederherstellen
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    print(filename)
    #filename = classes[i]
    #data = pickle.load(open(filename))
    data = pickle.load(open(filename,'rb'))
    target = [i] * len(data) #target sind quasi die label als integer, die postures, die werte in data sind die
    # angles der joints, gehören alle zu der Pose 'Knee' hier als [2] definiert
    
    return data, target
#print(target)
load_pose_data(2)

robot_pose_data/Knee


([[0.3037738800048828,
   0.015382050536572933,
   0.42035794258117676,
   1.9818861484527588,
   0.3037738800048828,
   -0.01683211326599121,
   0.5199840068817139,
   1.7119860649108887,
   0.0,
   -0.08726692199707031],
  [0.3037738800048828,
   0.015382050536572933,
   0.42035794258117676,
   1.9818861484527588,
   0.3037738800048828,
   -0.01683211326599121,
   0.5199840068817139,
   1.7119860649108887,
   0.0,
   -0.08726692199707031],
  [0.3037738800048828,
   0.013848066329956055,
   0.42035794258117676,
   1.9818861484527588,
   0.3037738800048828,
   -0.01683211326599121,
   0.5199840068817139,
   1.7119860649108887,
   0.0,
   -0.08726692199707031],
  [0.3037738800048828,
   0.015382050536572933,
   0.42035794258117676,
   1.9818861484527588,
   0.3037738800048828,
   -0.01683211326599121,
   0.5199840068817139,
   1.7119860649108887,
   0.0,
   -0.08726692199707031],
  [0.3037738800048828,
   0.015382050536572933,
   0.42035794258117676,
   1.9818861484527588,
   0.30377388

In [57]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE

for i in range(11):
    #all_data.append(load_pose_data(i)[0]) #die 0 nimmt nur den 1. return wert
    #all_target.append(load_pose_data(i)[1]) # append erzeugt array in array
    all_data += load_pose_data(i)[0]
    all_target += load_pose_data(i)[1]

print('total number of data', len(all_data))
#print(all_data)
print(all_target)

robot_pose_data/Crouch
robot_pose_data/Crouch
robot_pose_data/Belly
robot_pose_data/Belly
robot_pose_data/Knee
robot_pose_data/Knee
robot_pose_data/Frog
robot_pose_data/Frog
robot_pose_data/HeadBack
robot_pose_data/HeadBack
robot_pose_data/Back
robot_pose_data/Back
robot_pose_data/Left
robot_pose_data/Left
robot_pose_data/StandInit
robot_pose_data/StandInit
robot_pose_data/Stand
robot_pose_data/Stand
robot_pose_data/Right
robot_pose_data/Right
robot_pose_data/Sit
robot_pose_data/Sit
total number of data 222
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 

In [58]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]
#print(permutation)
print(n_training_data)
print(training_data)

[ 97 120  27 161  93  57 127 191  25  48 194  26  31 158  19 155  33 182
 140  63 136 165  39 119  37  36 132 160   6 121 134  86  28 190 110 175
 173  56  42  91 123 107   1 183 150 153 196 211  52   2   7 144 146 148
 174  88  12 217 193  50 139 147 212 118 115 209 178 106 104  89 198  43
 210 201  59 133  17 203 102  44 208  95  53  85 116 112  24  34 137 181
 202 156 177 122 162 108  55 197   0 168  61  11 179 124  71 142 188  84
 114  10  60 163  40 218   9 138 143  80  82  73  46  21  72 170  47  83
 207  23 220  81  68   5  54 154  76  62  99  51 215  77  65  22 185 130
 125  94  41  35 189  98 126 111 164  32  18  20 145  45 113  69 205 100
 221 109  74   8 135 159  30 105 131 171  67  64 167 117  66 199  75 149
 166 176  15  49  79 204  92   4 206 151   3 216  70 184  78  29 157 192
 200 214 180 195 169  38  58 141 152  87 101 187 128  14  13 172 103 129
  16 219  90  96 186 213]
155
[ 97 120  27 161  93  57 127 191  25  48 194  26  31 158  19 155  33 182
 140  63 136 165  39 

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [59]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [60]:
# YOUR CODE HERE
#clf.fit(digits.data[:-500], digits.target[:-500]) 

all_data=np.array(all_data)
all_target=np.array(all_target)
tr_data = np.array(training_data)

print(type(tr_data))
print(type(tr_data[1]))

clf.fit(all_data[tr_data],all_target[tr_data]) #für indexing muss das index array ein np.array sein
# und auch das 'äußere array'

#print(all_data[tr_data)
print(tr_data)
print(all_data[-1])

<class 'numpy.ndarray'>
<class 'numpy.int64'>
[ 97 120  27 161  93  57 127 191  25  48 194  26  31 158  19 155  33 182
 140  63 136 165  39 119  37  36 132 160   6 121 134  86  28 190 110 175
 173  56  42  91 123 107   1 183 150 153 196 211  52   2   7 144 146 148
 174  88  12 217 193  50 139 147 212 118 115 209 178 106 104  89 198  43
 210 201  59 133  17 203 102  44 208  95  53  85 116 112  24  34 137 181
 202 156 177 122 162 108  55 197   0 168  61  11 179 124  71 142 188  84
 114  10  60 163  40 218   9 138 143  80  82  73  46  21  72 170  47  83
 207  23 220  81  68   5  54 154  76  62  99  51 215  77  65  22 185 130
 125  94  41  35 189  98 126 111 164  32  18]
[-0.31289411 -0.23005795 -1.67048407  0.96637797 -0.31289411  0.03532386
 -1.62280262  0.82993603  0.02531069 -0.16758746]


### predicting

In [72]:
#print(all_data)
#all_data2 = all_data[:,np.newaxis]
#for i in range(len(all_data)):
    #all_data[i]=all_data[i][:,np.newaxis]
#print(shape(all_data2))
#print(shape(all_data2[1]))
#all_data.reshape(1,-1)
#print(all_data)
print(all_data[-1])
clf.predict([all_data[-1]]), all_target[-1]

[-0.31289411 -0.23005795 -1.67048407  0.96637797 -0.31289411  0.03532386
 -1.62280262  0.82993603  0.02531069 -0.16758746]


(array([10]), 10)

In [73]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [82]:
expected = []
predicted = []
# YOUR CODE HERE
index_test = permutation[n_training_data:]
pred_test = clf.predict(all_data[index_test])
target_test = all_target[index_test]

print(index_test)
expected.extend(target_test)
predicted.extend(pred_test)

print(expected)
print(predicted)


evaluate(expected, predicted)

[ 20 145  45 113  69 205 100 221 109  74   8 135 159  30 105 131 171  67
  64 167 117  66 199  75 149 166 176  15  49  79 204  92   4 206 151   3
 216  70 184  78  29 157 192 200 214 180 195 169  38  58 141 152  87 101
 187 128  14  13 172 103 129  16 219  90  96 186 213]
[0, 7, 1, 6, 4, 10, 5, 10, 6, 4, 0, 7, 7, 1, 6, 7, 7, 3, 3, 7, 6, 3, 10, 4, 7, 7, 8, 0, 2, 5, 10, 5, 0, 10, 7, 0, 10, 4, 8, 4, 0, 7, 9, 10, 10, 8, 9, 7, 1, 2, 7, 7, 5, 5, 9, 7, 0, 0, 7, 6, 7, 0, 10, 5, 5, 9, 10]
[0, 7, 1, 6, 4, 10, 5, 10, 6, 4, 0, 7, 7, 1, 6, 7, 7, 3, 3, 7, 6, 3, 10, 4, 7, 7, 8, 0, 2, 5, 10, 5, 0, 10, 7, 0, 10, 4, 8, 4, 0, 7, 9, 10, 10, 8, 0, 7, 1, 2, 7, 7, 5, 5, 9, 7, 0, 0, 7, 6, 7, 0, 10, 5, 5, 9, 10]
Classification report:
              precision    recall  f1-score   support

           0       0.90      1.00      0.95         9
           1       1.00      1.00      1.00         3
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         3
         

## 4. Evaluate on the test data

In [84]:
expected = []
predicted = []
# YOUR CODE HERE

index_test = permutation[n_training_data:]
pred_test = clf.predict(all_data[index_test])
target_test = all_target[index_test]

print(index_test)
expected.extend(target_test)
predicted.extend(pred_test)

print(expected)
print(predicted)

evaluate(expected, predicted)

[ 20 145  45 113  69 205 100 221 109  74   8 135 159  30 105 131 171  67
  64 167 117  66 199  75 149 166 176  15  49  79 204  92   4 206 151   3
 216  70 184  78  29 157 192 200 214 180 195 169  38  58 141 152  87 101
 187 128  14  13 172 103 129  16 219  90  96 186 213]
[0, 7, 1, 6, 4, 10, 5, 10, 6, 4, 0, 7, 7, 1, 6, 7, 7, 3, 3, 7, 6, 3, 10, 4, 7, 7, 8, 0, 2, 5, 10, 5, 0, 10, 7, 0, 10, 4, 8, 4, 0, 7, 9, 10, 10, 8, 9, 7, 1, 2, 7, 7, 5, 5, 9, 7, 0, 0, 7, 6, 7, 0, 10, 5, 5, 9, 10]
[0, 7, 1, 6, 4, 10, 5, 10, 6, 4, 0, 7, 7, 1, 6, 7, 7, 3, 3, 7, 6, 3, 10, 4, 7, 7, 8, 0, 2, 5, 10, 5, 0, 10, 7, 0, 10, 4, 8, 4, 0, 7, 9, 10, 10, 8, 0, 7, 1, 2, 7, 7, 5, 5, 9, 7, 0, 0, 7, 6, 7, 0, 10, 5, 5, 9, 10]
Classification report:
              precision    recall  f1-score   support

           0       0.90      1.00      0.95         9
           1       1.00      1.00      1.00         3
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         3
         

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [85]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'wb')) #hier muss 'wb' stehen statt nur 'w' , b für binary

Then, in the application we can load the trained classifier again.

In [88]:
clf2 = pickle.load(open(ROBOT_POSE_CLF,'rb')) #hier 'rb' statt gar nichts
clf2.predict([all_data[-1]]), all_target[-1] # hier wieder eckige klammern um all_data[-1] da es liste von arrays sein muss und nicht nur ein element

(array([10]), 10)